## ENV

__Source__: http://usda.mannlib.cornell.edu/MannUsda/viewDocumentInfo.do?documentID=1048

__Not using__: https://pypi.python.org/pypi/nass/0.1.0

__Approach__: Not using the CSV files because text file processing is more interesting, and we do not need to download 

__Python__: 3

__Libraries__: urllib3, pandas, re, regex, sys, BeautifulSoup4

__Date__: 08-Feb-2018

__Github__: The code is available in my Github as well along with these comments to avoid plagiarism


## NOTE/ DISCLAIMER

This is not a production scale work and the code has been run for a few years from 2015 until 2017. There are a lot of testing that needs to be done

This program does not require downloading any files and runs by downloading all the text files. This program is designed for this particular data set and conditions only. In case the file sizes are large then the algorithms will have to change


## PENDING ITEMS

1. The conversion of data types for dates and numerical values in the data frame needs to be completed
2. Documentation and comments
3. Data Quality Checks 
4. Test Cases for parsing and checks



## LICENSE 

Gourav Sengupta. This is for evaluation and not for production use.

In [46]:
import urllib3
import pandas
import re, regex
import sys
from bs4 import BeautifulSoup

In [47]:
pandas.options.display.max_rows = 999

__Plantation Results__

A sample of the table which is concatenated across for the entire year into the single dataframe:

```
Sunflowers Harvested - Selected States
[These 4 States harvested 84% of the 2015 sunflower acreage]
------------------------------------------------------------------------
                :               Week ending               :             
                :-----------------------------------------:             
      State     :November 27, :November 20, :November 27, :  2011-2015  
                :    2015     :    2016     :    2016     :   Average   
------------------------------------------------------------------------
                :                        percent                        
                :                                                       
Colorado .......:     93            92            97            96      
Kansas .........:     94            89            95            94      
North Dakota ...:     99            92            97            94      
South Dakota ...:     99            97            99            95      
                :                                                       
4 States .......:     98            94            98            94      
------------------------------------------------------------------------
```

In [33]:

def mainTableInstanceFromPage(resultParse):
    #crop 
    cropType = resultParse[0].strip()
    #the entire table
    theTable = regex.match("(Week\sending.*\r\n.*\r\n.*(:\s{0,}[a-zA-Z]*\s+\d{1,},\s{0,}){1}.*\r\n.*(:\s+\d\d\d\d\s+){3}|\sState\s+(:\s?[a-zA-Z\s]+){3,5}).*\r\n---+\r\n.*percent.*\r\n.\s+:\s+\r\n(([a-zA-Z\s]+\s\.+:)(\s+\d+|\s+\-|\s+\(NA\))+.*\r\n)+",resultParse[1]).captures(5)
    #preparing the date headings
    datePart1 = regex.findall("(:\s{0,}[a-zA-Z]+\s+\d+,\s{0,})",resultParse[1])
    datePart1 = [" ".join(h.replace(":","").split()[-2:]) for h in datePart1]
    datePart2 = regex.findall("(:\s+\d\d\d\d\s+)",resultParse[1])
    datePart2 = [h.split()[1] for h in datePart2]
    datePart = [" ".join([datePart1[h], datePart2[h]]) for h in range(0,len(datePart2))]
    #creating the pandas frame
    dfCheck = pandas.DataFrame([regex.split(":|\s\s+|\s\.+",h.rstrip()) for h in theTable], columns=["state","dummy","dummy1"]+datePart+["dummy2"])
    resultFrame = dfCheck.filter(items=["state"] + datePart).melt(id_vars=["state"]).rename(columns={"variable":"weekEnding","value":"metricsPercent"})
    resultFrame["metricsOf"] = cropType.strip()
    return resultFrame

def concatAllMainTableForPage(link):
    regularExpr = "([a-zA-Z\s]+)-.*\r\n\[.*\]\r\n---+\r\n.*(((Week\sending.*\r\n.*\r\n.*(:\s{0,}[a-zA-Z]*\s+\d{1,},\s{0,}){1}.*\r\n.*(:\s+\d\d\d\d\s+){3}).*\r\n---+\r\n.*percent.*\r\n.\s+:\s+\r\n(([a-zA-Z\s]+\s\.+:)(\s+\d+|\s+\-)+.*\r\n)+)+)"
    try: 
        http = urllib3.PoolManager()
        resultTest = regex.findall(regularExpr, http.request('GET', link, retries=False).data.decode('ascii'))
        http.clear()
        return pandas.concat(([mainTableInstanceFromPage(h) for h in resultTest]))
    except:
        http.clear()
        print("link: {}".format(link))
        print("Error:", sys.exc_info()[0])

__Plantation Conditions__

A sample of the table which is concatenated across for the entire year:

```
Winter Wheat Condition - Selected States: Week Ending November 27, 2016
[National crop conditions for selected States are weighted based on 2015
planted acreage]
----------------------------------------------------------------------------
      State     : Very poor :   Poor    :   Fair    :   Good    : Excellent 
----------------------------------------------------------------------------
                :                          percent                          
                :                                                           
Arkansas .......:     5          10          35          40          10     
California .....:     -           -           -          25          75     
Colorado .......:     3          12          38          41           6     
Idaho ..........:     -           -          22          55          23     
Illinois .......:     3           2          25          56          14     
Indiana ........:     2           4          26          53          15     
Kansas .........:     3          10          35          45           7     
Michigan .......:     2           3          27          52          16     
Missouri .......:     3           6          33          51           7     
Montana ........:     -           2          21          49          28     
Nebraska .......:     2          11          34          48           5     
North Carolina .:     1          12          29          56           2     
Ohio ...........:     -           1          12          61          26     
Oklahoma .......:     3           9          35          46           7     
Oregon .........:     1           3          14          65          17     
South Dakota ...:     1           8          40          46           5     
Texas ..........:     4          12          43          34           7     
Washington .....:     -           1           8          74          17     
                :                                                           
18 States ......:     2           8          32          47          11     
                :                                                           
Previous week ..:     2           8          32          47          11     
Previous year ..:     2           7          36          44          11     
----------------------------------------------------------------------------
```

In [79]:


def conditionTableInstanceFromPage(resultParse):
    # crop type 
    cropType = resultParse[0].strip()
    # weekending 
    weekEnding = resultParse[1]
    # preparing the headings
    headings = [h.strip() for h in resultParse[5].split(":")]
    # the entire table
    theTable = regex.match("(\s?State\s+(:\s?[a-zA-Z\s]+){3,}).*\r\n---+\r\n.*percent.*\r\n.\s+:\s+\r\n(([a-zA-Z\s]+\s\.+:)(\s+\d+|\s+\-)+.*\r\n)+",resultParse[3]).captures(3)    
    dfCheck = pandas.DataFrame([regex.split(":|\s\s+|\s\.+",h.rstrip()) for h in theTable], columns=["state","dummy","dummy1"] + headings[1:])
    resultFrame = dfCheck.filter(items=["state"] + headings[1:]).melt(id_vars=["state"]).rename(columns={"value":"metricsPercent"})
    resultFrame["measurementOf"] = cropType
    resultFrame["WeekEnding"] = weekEnding
    return resultFrame
    
def concatAllConditionTableForPage(link):
    regularExpr = "([a-zA-Z\s]+)-.*:\s+Week\sEnding\s+(.*).*\r\n\[(.*\r\n)?.*\]\r\n---+\r\n.*(((State\s+(:\s?[a-zA-Z\s]+)+).*\r\n---+\r\n.*percent.*\r\n.\s+:\s+\r\n(([a-zA-Z\s]+\s\.+:)(\s+\d+|\s+\-)+.*\r\n)+)+)"
    try: 
        http = urllib3.PoolManager()
        resultTest = regex.findall(regularExpr, http.request('GET', link, retries=False).data.decode('ascii'))
        http.clear()
        return pandas.concat(([conditionTableInstanceFromPage(h) for h in resultTest]))
    except:
        http.clear()
        print("link: {}".format(link))
        print("Error:", sys.exc_info()[0])

# START PROGRAM

Enter value of the year below to find all the required results in a data frame for that year

In [70]:
yearValue = 2017

Fetching all the links for the particular year

In [80]:
http = urllib3.PoolManager()

link = "http://usda.mannlib.cornell.edu/MannUsda/viewDocumentInfo.do?documentID=1048"
document = http.request('GET', link, retries=False)

soup = BeautifulSoup(document.data, 'html.parser')
fileLinks = [h.contents[1].attrs["href"] for h in soup.find_all(id=re.compile(".*{}.txt".format(yearValue)))]

In [81]:
%time plantationTable = pandas.concat([concatAllMainTableForPage(eachLink) for eachLink in fileLinks])

CPU times: user 1.68 s, sys: 22.3 ms, total: 1.7 s
Wall time: 10.2 s


In [82]:
%time conditionTable = pandas.concat([concatAllConditionTableForPage(eachLink) for eachLink in fileLinks])

CPU times: user 1.27 s, sys: 22.8 ms, total: 1.29 s
Wall time: 9.71 s


In [86]:
plantationTable[:4]

,state,weekEnding,metricsPercent,metricsOf
0,Colorado,"November 26, 2016",99,Corn Harvested
1,Illinois,"November 26, 2016",100,Corn Harvested
2,Indiana,"November 26, 2016",99,Corn Harvested
3,Iowa,"November 26, 2016",99,Corn Harvested


In [85]:
conditionTable[10:14]

,state,variable,metricsPercent,measurementOf,WeekEnding
10,Nebraska,Very poor,2,Winter Wheat Condition,"November 26, 2017"
11,North Carolina,Very poor,-,Winter Wheat Condition,"November 26, 2017"
12,Ohio,Very poor,-,Winter Wheat Condition,"November 26, 2017"
13,Oklahoma,Very poor,2,Winter Wheat Condition,"November 26, 2017"


## COMMENTS

This needs to be completed

In [736]:
re.compile(r""" 
 
 (                                # Beginning of a table
 (                                # HEADER - START (This is for reading two different kind of headers)
 Week\sending.*                   # Week ending string
 \r\n.*\r\n.*                     # Two new lines with any characters following them 
 (:\s?[a-zA-Z]*\s+\d{1,},\s)+     # Matching for multiple occurrences of string of type ": October 2,"
 .*\r\n.*                         # Any characters, followed by nebw line, followed by any characters
 (:\s+\d\d\d\d\s+)+               # Matching for multiple occurrences of type ":    2015     "
 )                                # HEADER - END (Completion of heading parsing)
 .*\r\n---+                       # Matching for line following heading "------"
 \r\n.*percent.*\r\n.*            # The following line having the percent keyword followed by another line and strings
 .\s+:\s+\r\n                     # The line after the line containing the word percent
 (                                # Multiple matches of lines like "Arkansas .......:     7          10          26          43          14"
 ([a-zA-Z\s]+\s\.+:)              # Matching for State "Arkansas .......:"
 (\s+\d+|\s+\-)+                  # Matching for percent values
 .*\r\n                           # New line at end of the line containing the percent values
 )+                               # Multiple occurrences of the lines containing percent values
 )+                               # Ending of a table
""", re.VERBOSE)